In [15]:
import os
import pathlib
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
from IPython.display import display, HTML
display(HTML("<style>.jp-CodeCell.jp-mod-outputsScrolled .jp-Cell-outputArea { max-height: 24em; }</style>"))
import numpy as np
import tritonclient.http as httpclient
from tritonclient.utils import np_to_triton_dtype

In [16]:
from transformers import AutoTokenizer

In [17]:
def prepare_tensor(name, input):
    client_util = httpclient
    t = client_util.InferInput(name, input.shape,
                               np_to_triton_dtype(input.dtype))
    t.set_data_from_numpy(input)
    return t

In [23]:
triton_url = "localhost:8000"
triton_model_name = "ensemble"
triton_client = httpclient.InferenceServerClient(url=triton_url)

input0 = [["tailor swifdt"],  ["the beatlos"]]
input0_data = np.array(input0).astype(object)
max_new_tokens = 5
input1_data = np.ones_like(input0).astype(np.int32) * max_new_tokens
inputs = [
    prepare_tensor("text_input", input0_data),
    prepare_tensor("max_new_tokens", input1_data)
]

outputs = []
outputs.append(httpclient.InferRequestedOutput("text_output"))
results = triton_client.infer(model_name=triton_model_name,
                              inputs=inputs,
                              outputs=outputs)

outputs = results.as_numpy('text_output')
outputs = [o.decode() for o in outputs]
print(outputs)

['</s><s>taylor swift</s></s>', '</s><s>the beatles</s></s>']
